In [1]:
import pandas as pd

In [2]:
import nltk
import multiprocessing
from functools import partial
from nltk.tokenize import word_tokenize
import numpy as np
from tensorflow.keras.layers import LSTM, Dense, Embedding, RepeatVector
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import random
#from googletrans import Translator
from scipy.special import softmax

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
df_raw = pd.read_csv('/kaggle/input/dataset-recipes/RAW_recipes.csv')

print(f'Número de filas: {df_raw.shape[0]}, Número de columnas:{df_raw.shape[1]}')

Número de filas: 231637, Número de columnas:12


In [4]:
df_raw.columns

Index(['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags',
       'nutrition', 'n_steps', 'steps', 'description', 'ingredients',
       'n_ingredients'],
      dtype='object')

In [7]:
def check_ingredients_in_steps(ingredients, steps):
    for ingredient in ingredients:
        if ingredient not in ' '.join(steps):
            return False
    return True


In [8]:
df_raw['all_ingredients_in_steps'] = df_raw.apply(lambda row: check_ingredients_in_steps(row['ingredients'], row['steps']), axis=1)
count = df_raw['all_ingredients_in_steps'].sum()
print(f'Número de registros en los que todos los ingredientes están incluidos en los pasos: {count}')


Número de registros en los que todos los ingredientes están incluidos en los pasos: 150793


In [9]:
filtered_df = df_raw[df_raw['all_ingredients_in_steps']]

In [10]:
df = filtered_df[:10000]
len(df)

10000

**Tokenizacion**

In [11]:
#Definimos una función para tokenizar una lista de texto (ingredientes o pasos)
def tokenize_list(text_list):
    # Removemos los corchetes y comillas del texto y luego dividimos por la coma
    return [ingredient.lower().strip().strip("'") for ingredient in text_list.strip("[]").split(",")]


df['ingredient_tokens'] = df['ingredients'].apply(tokenize_list)
df['steps_tokens'] = df['steps'].apply(tokenize_list)



/tmp/ipykernel_28/2618552342.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ingredient_tokens'] = df['ingredients'].apply(tokenize_list)
/tmp/ipykernel_28/2618552342.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['steps_tokens'] = df['steps'].apply(tokenize_list)


In [10]:
pd.set_option('display.max_colwidth', None)
print(df.loc[:1, 'ingredients'])
print(df.loc[:1, 'ingredient_tokens'])

print(df.loc[:1, 'steps'])
print(df.loc[:1, 'steps_tokens'])


0    ['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']
1           ['prepared pizza crust', 'sausage patty', 'eggs', 'milk', 'salt and pepper', 'cheese']
Name: ingredients, dtype: object
0    [winter squash, mexican seasoning, mixed spice, honey, butter, olive oil, salt]
1         [prepared pizza crust, sausage patty, eggs, milk, salt and pepper, cheese]
Name: ingredient_tokens, dtype: object
0    ['make a choice and proceed with recipe', 'depending on size of squash , cut into half or fourths', 'remove seeds', 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece', 'season with mexican seasoning mix ii', 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece', 'season with sweet mexican spice mix', 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin', 'be careful not to burn the squash especially if you op

**Creacion de Vocabulario**

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Creación del vocabulario
all_texts = df['ingredient_tokens'].tolist() + df['steps_tokens'].tolist()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_texts)

### Creacion de secuencias numericas

In [13]:
# Creación de secuencias numéricas
ingredient_sequences = tokenizer.texts_to_sequences(df['ingredient_tokens'])
steps_sequences = tokenizer.texts_to_sequences(df['steps_tokens'])


### Padding

In [14]:
# Padding
max_sequence_length = max(max(len(seq) for seq in ingredient_sequences), max(len(seq) for seq in steps_sequences))
ingredient_sequences = pad_sequences(ingredient_sequences, maxlen=max_sequence_length, padding='post')
steps_sequences = pad_sequences(steps_sequences, maxlen=max_sequence_length, padding='post')

### División de datos

In [17]:
# División de datos en conjuntos de entrenamiento, validación y prueba

train_size = int(0.8 * len(df_raw))
val_size = int(0.1 * len(df_raw))
train_ingredients = ingredient_sequences[:train_size]
train_steps = steps_sequences[:train_size]
val_ingredients = ingredient_sequences[train_size:train_size+val_size]
val_steps = steps_sequences[train_size:train_size+val_size]
test_ingredients = ingredient_sequences[train_size+val_size:]
test_steps = steps_sequences[train_size+val_size:]


# También puedes obtener el vocabulario creado por el Tokenizer y los índices de palabras utilizando:
word_index = tokenizer.word_index

# **Contrucción del Modelo y Training**

In [18]:
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Dense
import numpy as np

# Creación del vocabulario
all_texts = df['ingredient_tokens'].tolist() + df['steps_tokens'].tolist()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_texts)

# Verificar si '<start>' y '<end>' están presentes en el vocabulario, si no, agrégales manualmente
if '<start>' not in tokenizer.word_index:
    tokenizer.word_index['<start>'] = len(tokenizer.word_index) + 1
    tokenizer.index_word[len(tokenizer.word_index)] = '<start>'

if '<end>' not in tokenizer.word_index:
    tokenizer.word_index['<end>'] = len(tokenizer.word_index) + 1
    tokenizer.index_word[len(tokenizer.word_index)] = '<end>'

# Obtener el tamaño del vocabulario del tokenizer
vocab_size = len(tokenizer.word_index) + 1

# Asegurar que la longitud máxima de ingredientes y pasos sea la misma
max_sequence_length = 200

# Rellenar secuencias con ceros para que todas tengan la misma longitud
train_ingredients_padded = pad_sequences(train_ingredients, maxlen=max_sequence_length, padding='post')
train_steps_padded = pad_sequences(train_steps, maxlen=max_sequence_length, padding='post')

# Definir la arquitectura del modelo codificador-decodificador
embedding_dim = 100
rnn_units = 128

encoder_input = Input(shape=(max_sequence_length,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_input)
encoder_lstm = LSTM(rnn_units)(encoder_embedding)

decoder_input = Input(shape=(max_sequence_length,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_input)
decoder_lstm = LSTM(rnn_units, return_sequences=True)(decoder_embedding)
decoder_output = Dense(vocab_size, activation='softmax')(decoder_lstm)

# Crear el modelo codificador-decodificador
model = Model([encoder_input, decoder_input], decoder_output)

# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Ajustar la forma de train_steps_padded
train_steps_padded = np.pad(train_steps_padded, [(0, 0), (0, 1)], mode='constant', constant_values=0)

# Entrenar el modelo con más épocas (por ejemplo, 50 épocas)
#model.fit([train_ingredients_padded, train_steps_padded[:, :-1]], train_steps_padded[:, 1:], epochs=50, batch_size=32)

model.fit([train_ingredients_padded, train_steps_padded[:, :-1]], train_steps_padded[:, 1:], epochs=15, batch_size=32)



Epoch 1/15
313/313 [==============================] - 196s 598ms/step - loss: 2.0023 - accuracy: 0.9188
Epoch 2/15
313/313 [==============================] - 188s 602ms/step - loss: 0.9013 - accuracy: 0.9218
Epoch 3/15
313/313 [==============================] - 190s 606ms/step - loss: 0.8782 - accuracy: 0.9219
Epoch 4/15
313/313 [==============================] - 189s 605ms/step - loss: 0.8608 - accuracy: 0.9220
Epoch 5/15
313/313 [==============================] - 190s 607ms/step - loss: 0.8391 - accuracy: 0.9221
Epoch 6/15
313/313 [==============================] - 190s 606ms/step - loss: 0.8153 - accuracy: 0.9222
Epoch 7/15
313/313 [==============================] - 190s 607ms/step - loss: 0.7868 - accuracy: 0.9223
Epoch 8/15
313/313 [==============================] - 190s 608ms/step - loss: 0.7550 - accuracy: 0.9224
Epoch 9/15
313/313 [==============================] - 190s 608ms/step - loss: 0.7219 - accuracy: 0.9226
Epoch 10/15
313/313 [==============================] - 190s 608m

# **Guardar y descargar el modelo y el tokenizer**

In [19]:

model.save('/kaggle/working/modelo_token_words.h5')

In [20]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

In [21]:
download_file('/kaggle/working/modelo_token_words.h5', 'model')

/kaggle/working/model.zip

In [23]:
import pickle
# Guardar el tokenizer
with open('/kaggle/working/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)


In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 200)]        0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 200, 100)     12320300    ['input_2[0][0]']                
                                                                                                  
 lstm_1 (LSTM)                  (None, 200, 128)     117248      ['embedding_1[0][0]']            
                                                                                                  
 input_1 (InputLayer)           [(None, 200)]        0           []                               
                                                                                              

In [25]:
# para poder imprimir en párrafos
!pip install textwrap3

# **Generación de Recetas con ingredientes aleatorios**

In [39]:
import textwrap
def generate_recipe_from_random_ingredients(model, tokenizer, df, num_ingredients=5, max_steps=10, end_token='<end>', temperature=0.8):
    # Obtener la lista completa de tokens de ingredientes
    all_ingredients = df['ingredient_tokens'].tolist()
    all_tokens = [token for ingredients in all_ingredients for token in ingredients]

    # Seleccionar 5 tokens aleatorios
    random_tokens = random.sample(all_tokens, num_ingredients)

    print("Ingredientes seleccionados:")
    for i, token in enumerate(random_tokens):
        english_ingredient = token
        print(f"Ingrediente {i+1}: {english_ingredient}")




    start_token = '<start>'
    step_sequence = [tokenizer.word_index[start_token]]

    # Convertir los tokens seleccionados a secuencias numéricas utilizando el tokenizer
    random_tokens_sequence = tokenizer.texts_to_sequences([random_tokens])[0]

    # Encontrar la receta más larga entre las seleccionadas y usarla como longitud máxima
    max_recipe_length = min(max_steps, len(random_tokens_sequence) + 1)
    max_recipe_length = 200

    # Rellenar o truncar la secuencia de ingredientes para que tenga la misma longitud que max_recipe_length
    initial_ingredients_list = pad_sequences([random_tokens_sequence], maxlen=max_recipe_length , padding='post', truncating='post')
    

    # Generar los pasos de la receta
    for _ in range(max_steps - 1):
        step_sequence_input = pad_sequences([step_sequence], maxlen=max_recipe_length, padding='post', truncating='post')
        #next_token_probs = model.predict([initial_ingredients_list, step_sequence_input])[0, len(step_sequence_input[0]) - 1]
        next_token_probs = model.predict([initial_ingredients_list, step_sequence_input])[0, -1]

        # Utilizar temperatura para suavizar la distribución de probabilidad
        next_token_probs = softmax(next_token_probs)
        next_token_index = np.random.choice(len(next_token_probs), p=next_token_probs)

        # Verificar que el índice generado esté dentro del rango válido del vocabulario
        while next_token_index == 0 or tokenizer.index_word.get(next_token_index) is None:
            next_token_index = np.random.choice(len(next_token_probs), p=next_token_probs)

        next_token = tokenizer.index_word[next_token_index]
        step_sequence.append(next_token_index)

        if next_token == end_token:
            break

    # Convertir los índices de las palabras a texto, excluyendo el token de inicio
    recipe_text = ' '.join(tokenizer.index_word.get(idx, '') for idx in step_sequence[1:])

    
    return recipe_text

# Utilizando la función para generar una receta con 5 ingredientes aleatorios
recipe = generate_recipe_from_random_ingredients(model, tokenizer, df, num_ingredients=5, max_steps=8)
# Imprimir la receta con wordwrap
print("\nReceta generada:")
wrapper = textwrap.TextWrapper(width=80)
lines = wrapper.wrap(recipe)
for line in lines:
    print(line)
print('\n')




Ingredientes seleccionados:
Ingrediente 1: olive oil
Ingrediente 2: water
Ingrediente 3: crystallized ginger
Ingrediente 4: sugar
Ingrediente 5: raisins
1/1 [==============================] - 0s 26ms/step

Receta generada:
slice brisket across grain and pour remaining barbecue sauce over meat raisin
gather up the edges of the brie in place of flour use 2 cups rye cover and
simmer on low heat for 20 minutes egg bread and bacon over pitas and sprinkle
with cheese




In [40]:
# Utilizando la función para generar una receta con 5 ingredientes aleatorios
recipe = generate_recipe_from_random_ingredients(model, tokenizer, df, num_ingredients=5, max_steps=8)
# Imprimir la receta con wordwrap
print("\nReceta generada:")
wrapper = textwrap.TextWrapper(width=80)
lines = wrapper.wrap(recipe)
for line in lines:
    print(line)
print('\n')



Ingredientes seleccionados:
Ingrediente 1: red pepper
Ingrediente 2: tart apples
Ingrediente 3: butter
Ingrediente 4: coconut milk
Ingrediente 5: raisins
1/1 [==============================] - 0s 23ms/step

Receta generada:
food grinder or processor sprinkle the dried bread crumbs over the top heat oil
in a large pan and quickly pour the cornmeal batter into the skillet pour out
the drippings and wipe clean chop the walnuts or until cake test done




In [41]:
# Utilizando la función para generar una receta con 5 ingredientes aleatorios
recipe = generate_recipe_from_random_ingredients(model, tokenizer, df, num_ingredients=3, max_steps=8)
# Imprimir la receta con wordwrap
print("\nReceta generada:")
wrapper = textwrap.TextWrapper(width=80)
lines = wrapper.wrap(recipe)
for line in lines:
    print(line)
print('\n')



Ingredientes seleccionados:
Ingrediente 1: orange juice
Ingrediente 2: butter
Ingrediente 3: egg
1/1 [==============================] - 0s 26ms/step

Receta generada:
bake at 400 degrees for 7-8 minutes fold in 1 / 2 cup of the apples place an
apple slice between each biscuit and around the outer edge of the baking dish
remove bones at 375 for 20 to 25 minutes dry oregano shake well and strain into
a pitcher or a chilled martini glass


